In [4]:
import sys
import csv
from collections import Counter
sys.path.append("../")
from agreement.k_free import multirater_kfree
from agreement.cohen_kappa import multikappa

In [5]:
tags_of_interest = ['qy',
                    'qw',
                    'qy^d',
                    'qw^d',
                    '^g',
                    'br',
                    'bh',
                    'qc',
                    'qr',
                    'ny',
                    'nn',
                    'ny^e',
                    'nn^e',
                    'na',
                    'no',
                    'sd-qw',
                   'other']  # adding 'other for not marking these as having these tags

tag_idx_map = {tag:i for i, tag in enumerate(tags_of_interest)}
print tag_idx_map

# a list of m lists (where m = number of utterances/intances) and k+1 wide (where k = number of tags of interest) and +1 is for the 'other'
file_name = '../../../Dropbox/QM_lecturer/phd_supervision/Shamila_Wakefield_Brock_001_01 (version 1)_MP_JH.csv'

{'qy^d': 2, 'qr': 8, 'qw^d': 3, 'qw': 1, 'no': 14, 'qy': 0, 'na': 13, 'nn^e': 12, 'bh': 6, 'sd-qw': 15, 'ny': 9, 'qc': 7, 'other': 16, 'ny^e': 11, 'br': 5, 'nn': 10, '^g': 4}


In [6]:
def kappa_agreement_for_tagset(tags_of_interest, file_name,tag_idx_map, debug=False):
    all_annotations = []
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        tag_occurences = Counter()
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                utterance_annotations = [0] * len(tags_of_interest)
                for row_idx in [8,9,14]:
                    tag = row[row_idx]
                    if tag not in tags_of_interest:
                        if row_idx in [9,14] and not tag == '':
                            if debug:
                                print 'Warning: tag', tag, 'not in tags of interest for annotator', row_idx
                        tag = 'other'
                    utterance_annotations[tag_idx_map[tag]] += 1
                    tag_occurences[tag]+=1
                #print utterance_annotations
                all_annotations.append(utterance_annotations)
                line_count += 1
    return multikappa(all_annotations,3), tag_occurences

In [7]:
k,counts = kappa_agreement_for_tagset(tags_of_interest, file_name, tag_idx_map)
print k[0]
print counts, sum(counts.values())

Kappa =  0.844023735519
z =  15.7390472868
PA =  0.920138888889
PE =  0.487991898148
0.844023735519
Counter({'other': 398, 'qw': 30, 'qy': 26, 'sd-qw': 26, 'br': 22, 'qc': 15, 'qy^d': 12, 'ny': 12, 'ny^e': 11, 'na': 8, 'nn^e': 6, 'no': 4, 'qw^d': 3, '^g': 2, 'nn': 1}) 576


In [8]:
all_results = []
for i in range(0, len(tags_of_interest)-1):
    tag =  tags_of_interest[i]
    print "TAG in focus:", tag
    new_tags_of_interest = [tag] + ['other']
    new_tag_idx_map = {tag:k for k, tag in enumerate(new_tags_of_interest)}
    k, counts = kappa_agreement_for_tagset(new_tags_of_interest, file_name, new_tag_idx_map)
    all_results.append([tags_of_interest[i], counts[tags_of_interest[i]], k[0]])
    print "*" * 30

TAG in focus: qy
Kappa =  0.758321678322
z =  4.04927247105
PA =  0.979166666667
PE =  0.913797260802
******************************
TAG in focus: qw
Kappa =  0.894505494505
z =  5.16817801088
PA =  0.989583333333
PE =  0.901258680556
******************************
TAG in focus: qy^d
Kappa =  0.659574468085
z =  2.33343521032
PA =  0.986111111111
PE =  0.959201388889
******************************
TAG in focus: qw^d
Kappa =  1.0
z =  1.74111901147
PA =  1.0
PE =  0.989637586806
******************************
TAG in focus: ^g
Kappa =  0.498257839721
z =  0.707098181608
PA =  0.996527777778
PE =  0.99307966821
******************************
TAG in focus: br
Kappa =  0.952740400394
z =  4.64606244474
PA =  0.996527777778
PE =  0.926528742284
******************************
TAG in focus: bh
Kappa =  0
z =  0
PA =  1.0
PE =  1.0
******************************
TAG in focus: qc
Kappa =  0.794652406417
z =  3.15993612411
PA =  0.989583333333
PE =  0.949273003472
******************************
T

In [9]:
all_results

[['qy', 26, 0.7583216783216817],
 ['qw', 30, 0.894505494505496],
 ['qy^d', 12, 0.659574468085108],
 ['qw^d', 3, 1.0],
 ['^g', 2, 0.498257839721253],
 ['br', 22, 0.9527404003938301],
 ['bh', 0, 0],
 ['qc', 15, 0.7946524064171148],
 ['qr', 0, 0],
 ['ny', 12, 1.0],
 ['nn', 1, -0.0017391304347679584],
 ['ny^e', 11, 0.9073209975864847],
 ['nn^e', 6, 0.6631578947368483],
 ['na', 8, 0.8732394366197138],
 ['no', 4, 0.49650349650348563],
 ['sd-qw', 26, 0.6374825174825213]]

In [10]:
# check out the 'other' tag accuracy
debug = False
all_annotations = []
new_tags_of_interest = ['other', 'nonother']
new_tag_idx_map = {tag:k for k, tag in enumerate(new_tags_of_interest)}
with open(file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    tag_occurences = Counter()
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            utterance_annotations = [0] * len(new_tags_of_interest)
            for row_idx in [8,9,14]:
                tag = row[row_idx]
                if tag not in tags_of_interest:
                    if row_idx in [9,14] and not tag == '':
                        if debug:
                            print 'Warning: tag', tag, 'not in tags of interest for annotator', row_idx
                    tag = 'other'
                else:
                    tag = 'nonother'
                utterance_annotations[new_tag_idx_map[tag]] += 1
                tag_occurences[tag]+=1
            all_annotations.append(utterance_annotations)
            line_count += 1
print tag_occurences
print multikappa(all_annotations,3)

Counter({'other': 398, 'nonother': 178})
Kappa =  0.902433515894
z =  16.6939737923
PA =  0.958333333333
PE =  0.572940779321
(0.9024335158940783, 16.693973792276957, 0.0029222070088269062, 0.9583333333333338, 0.5729407793209876)
